In [2]:
#Mount drive to access files in gdrive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [4]:
# Import libraries
import pandas as pd
import numpy as np
import string
import re
import keras
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, Input
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras import backend as K
from keras.models import Model


In [5]:
# read train, test dev and augemented data
#df_train = pd.read_csv("/content/gdrive/MyDrive/Lakehead_NLP_Project/train.csv")
df_test = pd.read_csv("/content/gdrive/MyDrive/Lakehead_NLP_Project/dev.csv")
df_gold = pd.read_csv("/content/gdrive/MyDrive/Lakehead_NLP_Project/gold-test-27446.csv")
df_aug = pd.read_csv("/content/gdrive/MyDrive/Lakehead_NLP_Project/filtered_augemented_data.csv")

In [6]:
# text preprocessing
%%time
def text_preprocessing(text):
  text = text.lower()
  text = re.sub(r"http\S+", " ", text)
  text = re.sub(' +', ' ', text)
  
  return text

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs


In [7]:
# Applying pre-processing on all data
# train data
# df_train_1 = df_train.copy()
# df_train_1['text'] = df_train_1['text'].apply(text_preprocessing)

#test data
df_test_1 = df_test.copy()
df_test_1['text'] = df_test_1['text'].apply(text_preprocessing)

#gold data
df_gold_1 = df_gold.copy()
df_gold_1['text'] = df_gold_1['text'].apply(text_preprocessing)

#Augemented data
df_aug_1 = df_aug.copy()
df_aug_1['text'] = df_aug_1['text'].apply(text_preprocessing)

In [8]:
#intializing tokenizer, fitting it on train, test, dev and augemented data and then adding padding to each sequence

vocabulary_size = 20000
maxlen = 200

tokenizer = Tokenizer(num_words= vocabulary_size)

#creating vocabulary on train data
tokenizer.fit_on_texts(df_aug_1['text'])

#generating sequence of tokens on train, text, gold data
#train_sequences = tokenizer.texts_to_sequences(df_train_1['text'])
test_sequences = tokenizer.texts_to_sequences(df_test_1['text'])
gold_sequences = tokenizer.texts_to_sequences(df_gold_1['text'])
aug_sequences = tokenizer.texts_to_sequences(df_aug_1['text'])

#padding the sequences
#train_pad = pad_sequences(train_sequences, maxlen=maxlen)
test_pad = pad_sequences(test_sequences, maxlen=maxlen)
gold_pad = pad_sequences(gold_sequences, maxlen=maxlen)
aug_pad = pad_sequences(aug_sequences, maxlen=maxlen)

In [9]:
#reading glove embedding

def read_glove_embedding(vocabulary_size, df, glove_path, embedding_size, tokenizer):

  embeddings_index = dict()

  f = open(glove_path)
  for line in f:
      values = line.split()
      word = values[0]
      coefs = np.asarray(values[1:], dtype='float32')
      embeddings_index[word] = coefs
  f.close()
  print('Loaded %s word vectors.' % len(embeddings_index))

  # create a weight matrix for words in training docs
  embedding_matrix = np.zeros((vocabulary_size, embedding_size))
  for word, index in tokenizer.word_index.items():
      if index > vocabulary_size - 1:
          break
      else:
          embedding_vector = embeddings_index.get(word)
          if embedding_vector is not None:
              embedding_matrix[index] = embedding_vector

  return embedding_matrix


In [10]:
#create embedding for train data
vocabulary_size = 20000
df_new = df_aug_1
glove_path = '/content/gdrive/MyDrive/Lakehead_NLP_Project/glove.6B.100d.txt'
embedding_size = 100

embedding_matrix = read_glove_embedding(vocabulary_size, df_new, glove_path,  embedding_size, tokenizer)

Loaded 400000 word vectors.


In [11]:
# defining metrices to evaluate performance of model

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [12]:
#defining model
model_glove = Sequential()
model_glove.add(Embedding(vocabulary_size, 100, input_length=150, weights=[embedding_matrix], trainable=False))
model_glove.add(Dropout(0.2))
model_glove.add(Conv1D(64, 5, activation='relu'))
model_glove.add(MaxPooling1D(pool_size=4))
model_glove.add(LSTM(100))
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy',f1_m, precision_m, recall_m])
#plot_model(model_glove, to_file='model_plot.png', show_shapes=True, show_layer_names=True)


In [13]:
#summary of model
model_glove.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 150, 100)          2000000   
_________________________________________________________________
dropout (Dropout)            (None, 150, 100)          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 146, 64)           32064     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 36, 64)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               66000     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 2,098,165
Trainable params: 98,165
Non-trainable params: 2,000,000
_________________________________________

In [14]:
%%time
# fitting model on train data and validating on test data
model_glove.fit(aug_pad, np.array(df_aug_1['is_humor']), validation_data=(test_pad, np.asarray(df_test_1['is_humor'])), epochs = 10, batch_size=128)

Epoch 1/10
85/85 [==============================] - 51s 28ms/step - loss: 0.5564 - binary_accuracy: 0.7086 - f1_m: 0.7997 - precision_m: 0.7108 - recall_m: 0.9302 - val_loss: 0.4803 - val_binary_accuracy: 0.7730 - val_f1_m: 0.7998 - val_precision_m: 0.8920 - val_recall_m: 0.7257
Epoch 2/10
85/85 [==============================] - 1s 11ms/step - loss: 0.4321 - binary_accuracy: 0.8020 - f1_m: 0.8427 - precision_m: 0.8312 - recall_m: 0.8655 - val_loss: 0.4064 - val_binary_accuracy: 0.8130 - val_f1_m: 0.8547 - val_precision_m: 0.8283 - val_recall_m: 0.8852
Epoch 3/10
85/85 [==============================] - 1s 11ms/step - loss: 0.3395 - binary_accuracy: 0.8536 - f1_m: 0.8818 - precision_m: 0.8709 - recall_m: 0.8960 - val_loss: 0.3868 - val_binary_accuracy: 0.8120 - val_f1_m: 0.8513 - val_precision_m: 0.8391 - val_recall_m: 0.8660
Epoch 4/10
85/85 [==============================] - 1s 11ms/step - loss: 0.3033 - binary_accuracy: 0.8668 - f1_m: 0.8932 - precision_m: 0.8853 - recall_m: 0.9046 

In [15]:
# evaluating model on gold data
gold_lstm_results = model_glove.evaluate(gold_pad, np.asarray(df_gold_1['is_humor']), verbose=0, batch_size=128)


print(f'Gold Data accuracy: {gold_lstm_results[1]*100:0.2f}')
print(f'Gold Data F1 score: {gold_lstm_results[2]}')

Gold Data accuracy: 82.00
Gold Data F1 score: 0.853489875793457
